<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 22s 836ms/step - loss: 0.6763 - accuracy: 0.5596 - val_loss: 0.6651 - val_accuracy: 0.5463
Epoch 2/10
26/26 [==============================] - 20s 783ms/step - loss: 0.6226 - accuracy: 0.7068 - val_loss: 0.6004 - val_accuracy: 0.7366
Epoch 3/10
26/26 [==============================] - 20s 785ms/step - loss: 0.5568 - accuracy: 0.7579 - val_loss: 0.5325 - val_accuracy: 0.7317
Epoch 4/10
26/26 [==============================] - 20s 777ms/step - loss: 0.5215 - accuracy: 0.7372 - val_loss: 0.4815 - val_accuracy: 0.7512
Epoch 5/10
26/26 [==============================] - 20s 776ms/step - loss: 0.4491 - accuracy: 0.7968 - val_loss: 0.4448 - val_accuracy: 0.7951
Epoch 6/10
26/26 [==============================] - 21s 788ms/step - loss: 0.4123 - accuracy: 0.8248 - val_loss: 0.3766 - val_accuracy: 0.8098
Epoch 7/10
26/26 [==============================] - 21s 796ms/step - loss: 0.4018 - accuracy: 0.8345 - val_loss: 0.3631 - val_accuracy: 0.8976

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.62.h5
26/26 - 21s - loss: 0.6655 - accuracy: 0.5985 - val_loss: 0.6174 - val_accuracy: 0.7805
Epoch 2/5

Epoch 00002: saving model to weights.02-0.59.h5
26/26 - 21s - loss: 0.6031 - accuracy: 0.6922 - val_loss: 0.5946 - val_accuracy: 0.6976
Epoch 3/5

Epoch 00003: saving model to weights.03-0.54.h5
26/26 - 21s - loss: 0.5330 - accuracy: 0.7603 - val_loss: 0.5436 - val_accuracy: 0.6634
Epoch 4/5

Epoch 00004: saving model to weights.04-0.44.h5
26/26 - 21s - loss: 0.4604 - accuracy: 0.8236 - val_loss: 0.4433 - val_accuracy: 0.8195
Epoch 5/5

Epoch 00005: saving model to weights.05-0.41.h5
26/26 - 21s - loss: 0.3872 - accuracy: 0.8516 - val_loss: 0.4062 - val_accuracy: 0.8341


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6819 - accuracy: 0.5316 - val_loss: 0.6812 - val_accuracy: 0.4732


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 23s - loss: 0.6682 - accuracy: 0.5900 - val_loss: 0.6840 - val_accuracy: 0.4829
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 24s - loss: 0.6070 - accuracy: 0.6703 - val_loss: 0.5891 - val_accuracy: 0.7561


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 26s - loss: 0.6490 - accuracy: 0.6046 - val_loss: 0.7267 - val_accuracy: 0.4732
Epoch 2/50
26/26 - 27s - loss: 0.6036 - accuracy: 0.6983 - val_loss: 0.6656 - val_accuracy: 0.5171
Epoch 3/50
26/26 - 24s - loss: 0.5580 - accuracy: 0.7348 - val_loss: 0.5365 - val_accuracy: 0.7366
Epoch 4/50
26/26 - 23s - loss: 0.4734 - accuracy: 0.8017 - val_loss: 0.4292 - val_accuracy: 0.8244
Epoch 5/50
26/26 - 24s - loss: 0.4444 - accuracy: 0.7895 - val_loss: 0.4180 - val_accuracy: 0.8537
Epoch 6/50
26/26 - 22s - loss: 0.3701 - accuracy: 0.8564 - val_loss: 0.4924 - val_accuracy: 0.7561
Epoch 7/50
26/26 - 22s - loss: 0.3489 - accuracy: 0.8650 - val_loss: 0.3222 - val_accuracy: 0.8927
Epoch 8/50
26/26 - 22s - loss: 0.2674 - accuracy: 0.9173 - val_loss: 0.3218 - val_accuracy: 0.8829
Epoch 9/50
26/26 - 22s - loss: 0.2431 - accuracy: 0.9197 - val_loss: 0.2163 - val_accuracy: 0.9268
Epoch 10/50
26/26 - 22s - loss: 0.1731 - accuracy: 0.9586 - val_loss: 0.4550 - val_accuracy: 0.7902
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 817ms/step - loss: 0.6733 - accuracy: 0.5742 - val_loss: 0.6548 - val_accuracy: 0.6000
Epoch 2/5
26/26 [==============================] - 21s 790ms/step - loss: 0.6472 - accuracy: 0.6022 - val_loss: 0.6239 - val_accuracy: 0.7756
Epoch 3/5
26/26 [==============================] - 20s 773ms/step - loss: 0.5912 - accuracy: 0.7579 - val_loss: 0.6164 - val_accuracy: 0.5854
Epoch 4/5
26/26 [==============================] - 20s 777ms/step - loss: 0.5523 - accuracy: 0.7409 - val_loss: 0.5581 - val_accuracy: 0.7220
Epoch 5/5
26/26 [==============================] - 20s 769ms/step - loss: 0.5120 - accuracy: 0.7518 - val_loss: 0.4987 - val_accuracy: 0.8098


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.574209,0.673405,0.600000,0.654779
1,1,0.602190,0.646289,0.775610,0.623914
2,2,0.757908,0.591358,0.585366,0.616421
3,3,0.740876,0.553637,0.721951,0.558117
4,4,0.751825,0.512733,0.809756,0.498656


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 27s 1s/step - loss: 0.6813 - accuracy: 0.5535 - val_loss: 0.7424 - val_accuracy: 0.4341

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 21s 797ms/step - loss: 0.6622 - accuracy: 0.5912 - val_loss: 0.6628 - val_accuracy: 0.6098

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 21s 796ms/step - loss: 0.6490 - accuracy: 0.6448 - val_loss: 0.6593 - val_accuracy: 0.6049

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 21s 803ms/step - loss: 0.6422 - accuracy: 0.6533 - val_loss: 0.6565 - val_accuracy: 0.6049

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 778ms/step - loss: 0.6387

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 852ms/step - loss: 0.6755 - accuracy: 0.5584 - val_loss: 0.6659 - val_accuracy: 0.5463
Epoch 2/50
26/26 [==============================] - 21s 815ms/step - loss: 0.6394 - accuracy: 0.6533 - val_loss: 0.6215 - val_accuracy: 0.6000
Epoch 3/50
26/26 [==============================] - 21s 815ms/step - loss: 0.5869 - accuracy: 0.7287 - val_loss: 0.5833 - val_accuracy: 0.6439
Epoch 4/50
26/26 [==============================] - 20s 766ms/step - loss: 0.5357 - accuracy: 0.7421 - val_loss: 0.4929 - val_accuracy: 0.8341
Epoch 5/50
26/26 [==============================] - 20s 784ms/step - loss: 0.5171 - accuracy: 0.7628 - val_loss: 0.4573 - val_accuracy: 0.8390
Epoch 6/50
26/26 [==============================] - 20s 774ms/step - loss: 0.4432 - accuracy: 0.8114 - val_loss: 0.4286 - val_accuracy: 0.8439
Epoch 7/50
26/26 [==============================] - 20s 772ms/step - loss: 0.3923 - accuracy: 0.8321 - val_loss: 0.4116 - val_accuracy: 0.8537

In [ ]:
%tensorboard --logdir log_dir